# <a name="top_app"></a> Bloomberg Data Analytics: <br><span style="color:orange"> Using the Bloomberg Query Language with Equity Data: Advanced </span>

<a id='s0'></a>

#### Topics

1. [__Recap__](#s1)<a href='#s1'></a>
- [1.1 Filter](#s1.1)<a href='#s1.1'></a>
- [1.2 Custom calculations](#s1.2)<a href='#s1.2'></a>
2. [__Aggregating and Grouping Data in BQL__](#s2)<a href='#s2'></a>
- [2.1 Group](#s2.1)<a href='#s2.1'></a>
- [2.2 Filter & Group](#s2.2)<a href='#s2.2'></a>
- [2.3 Ungrouping with Groupxyz()](#s2.3)<a href='#s2.3'></a>
3. [__Advanced Features__](#s3)<a href='#s3'></a>
- [3.1 Quantile](#s3.1)<a href='#s3.1'></a>
- [3.2 Rolling calculations](#s3.2)<a href='#s3.2'></a>
- [3.3 Value](#s3.3)<a href='#s3.3'></a>
- [3.4 Advanced filtering](#s3.4)<a href='#s3.4'></a>
- [3.5 Matches](#s3.5)<a href='#s3.5'></a>
4. [__Scoring model__](#s4)<a href='#s4'></a>
- [4.1 Piotroski F-Score](#s4.1)<a href='#s4.1'></a>
- [4.2 Beneish M-Score](#s4.2)<a href='#s4.2'></a>
- [4.3 Scoring Practice](#s4.3)<a href='#s4.3'></a>

This tutorial assumes knowledge of BQL Basics for Equities such as standardization of fields, filter(), and members()


<a id='s1'></a>

<span style="color:darkorange; font-size:2em"> 1 Recap </span><br>
Let's import our packages and initiate the connection to BQL first

In [1]:
import bql
import pandas as pd
from collections import OrderedDict

In [2]:
bq = bql.Service()

<a id='s1.1'></a>
### 1.1 Filter

- Screen your universe directly on the Bloomberg Server

<img src="../../Visualisations/BQL Filtering.jpg" style="width: 500px;"/>

In [27]:
#lets only retreive the financials companies in the DOW Jones index
univ = bq.univ.members('INDU Index')
sector = bq.data.gics_sector_name() 
screen = bq.univ.filter(univ, sector == 'Financials')

#Define our request
req = bql.Request(screen, sector)
res = bq.execute(req)

data = res[0].df()
data

,GICS_SECTOR_NAME()
ID,
AXP UN Equity,Financials
JPM UN Equity,Financials
TRV UN Equity,Financials
GS UN Equity,Financials


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s1.2'></a>
### 1.2 Custom Calculations


One of the key benefits of BQL is that this gives you more flexibility with custom factors.
- Custom combination of fields defined by yourself
- Calculations and historical analysis of your custom fields
- Apply parameters to metrics eg technical indicators and returns calculations such as alpha and beta

See the BQL Editor for field explanations, parameters and defaults

##### <span style="color:darkorange">Example </span>: Stock upside potential for stocks

In [3]:
equity = 'ABBN SW Equity '
# Define Upside Potential
target = bq.data.target_price()
px = bq.data.px_last()
upside = (target/px - 1) * 100
#Shift + Tab after a data item shows the available parameters 

req = bql.Request(equity, {'Upside':upside})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Upside
ID,,,
ABBN SW Equity,2020-10-30,None,10.789372


In [4]:
#This means we can analyse this custom field historical time series
equity = 'ABBN SW Equity'

# Define Upside Potential
target = bq.data.target_price(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
px = bq.data.px_last(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
upside = (target/px - 1) * 100

req = bql.Request(equity, {'Upside':upside})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Upside
ID,,,
ABBN SW Equity,2020-10-23,None,3.776787
ABBN SW Equity,2020-10-24,None,3.424069
ABBN SW Equity,2020-10-25,None,3.488399
ABBN SW Equity,2020-10-26,None,4.361912
ABBN SW Equity,2020-10-27,None,5.108650
ABBN SW Equity,2020-10-28,None,10.294995
ABBN SW Equity,2020-10-29,None,11.440192
ABBN SW Equity,2020-10-30,None,10.789372


Calculations of this custom time series e.g. show the percentage change of the upside potential over the past week

In [5]:
#This also means we can peform analysis on this custom time series
equity = 'ABBN SW Equity'

# Define Upside Potential
target = bq.data.target_price(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
px = bq.data.px_last(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
upside = (target/px - 1) * 100
#calcuate the change
upside_chg = upside.pct_chg()

req = bql.Request(equity, {'Upside Change':upside_chg})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Upside Change
ID,,,
ABBN SW Equity,2020-10-30,None,185.675945


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2'></a>
<span style="color:darkorange; font-size:2em"> 2 Aggregating and Grouping Data in BQL </span>

<a id='s2.1'></a>

### 2.1 Group

- aggregating on Bloomberg Server
- using members() to access indices and portfolios

<img src="../../Visualisations/BQL Grouping.jpg" style="width: 800px;"/>

##### <span style="color:darkorange">Q </span>: Count the number of securities per BICS Level 1 sector of Stoxx 600

In [28]:
#Index Members
index = bq.univ.members('SXXP Index')
#Count Members by sector
security = bq.data.id()
sector = bq.data.classification_name('BICS', '1')
count_sector = security.group(sector).count()

req = bql.Request(index, count_sector)
res = bq.execute(req)
data = res[0].df()
data

,Weights,Positions,ORIG_IDS,"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.BICS,CLASSIFICATION_LEVEL.1)","COUNT(GROUP(ID(),CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.BICS,CLASSIFICATION_LEVEL.1)))"
ID,,,,,
Communications,NaN,NaN,SXXP INDEX,Communications,39
Consumer Discretionary,NaN,NaN,SXXP INDEX,Consumer Discretionary,56
Consumer Staples,NaN,NaN,SXXP INDEX,Consumer Staples,53
Energy,NaN,NaN,SXXP INDEX,Energy,21
Financials,NaN,NaN,SXXP INDEX,Financials,102
Health Care,NaN,NaN,SXXP INDEX,Health Care,53
Industrials,NaN,NaN,SXXP INDEX,Industrials,111
Materials,NaN,NaN,SXXP INDEX,Materials,60
Real Estate,NaN,NaN,SXXP INDEX,Real Estate,38


<a id='s2.2'></a>

### 2.1 Filter & Group


##### <span style="color:darkorange">Q </span>: Show the members of the SX5E whose price performance was positive and was greater than their sector average?

In [7]:
index = bq.univ.members('SX5E Index')

px_perf = bq.func.pct_chg(bq.data.px_last(dates=bq.func.range('-1m','0d'), fill='prev', ca_adj='full'))
sector_avg = bq.func.groupavg(px_perf, [bq.data.gics_sector_name()])
screen = bq.univ.filter(index, bq.func.and_(px_perf > sector_avg, px_perf > 0))


req = bql.Request(index, {'PX_PERF': sector_avg})
res = bq.execute(req)
data = res[0].df()
data.head()

,DATE,CURRENCY,ORIG_IDS,GICS_SECTOR_NAME(),PX_PERF
ID,,,,,
OR FP Equity,2020-10-30,None,None,Consumer Staples,-4.480457
DG FP Equity,2020-10-30,None,None,Industrials,-2.220257
ASML NA Equity,2020-10-30,None,None,Information Technology,-12.526999
SAN SQ Equity,2020-10-30,None,None,Financials,-7.161019
PHIA NA Equity,2020-10-30,None,None,Health Care,-11.209173


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2.3'></a>
### 2.3 Ungrouping with Groupxyz()



- Bucket level calculation with output at security level
- eg what is the rank of each security's rank within its own sector?

<img src="../../Visualisations/BQL Groupxyz.jpg" style="width: 800px;"/>

- The groupxyz functions make this easier by making the manipulation in the background.
- Common applications include grouprank(), groupzscore(), grouprank(), groupavg() and groupmedian()
- You can stack groupxyz() functions to manage your output within BQL.

##### <span style="color:darkorange">Example </span>: Show the top 10 members of the SMI index who increased upside potential the most in past week

In [29]:
#Index is our universe
index = bq.univ.members('SMI Index')
# Define Upside Potential
target = bq.data.target_price(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
px = bq.data.px_last(dates= bq.func.range('-1w','0d'), fill='prev', frq='d')
upside = (target/px - 1) * 100
upside_chg = upside.pct_chg()
#Define our filter and apply
criteria = upside_chg.grouprank() <=10
filtered_index = bq.univ.filter(index, criteria)

req = bql.Request(filtered_index, {'Top 10 Upside Change':upside_chg.groupsort()})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Top 10 Upside Change
ID,,,
ALC SE Equity,2020-10-30,None,1189.691133
SGSN SE Equity,2020-10-30,None,272.878578
CFR SE Equity,2020-10-30,None,243.558490
ABBN SE Equity,2020-10-30,None,184.358704
UHR SE Equity,2020-10-30,None,137.597719
SIKA SE Equity,2020-10-30,None,90.982134
CSGN SE Equity,2020-10-30,None,69.915620
SCMN SE Equity,2020-10-30,None,61.129314
SLHN SE Equity,2020-10-30,None,52.238238


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s3'></a>

<span style="color:darkorange; font-size:2em"> 3 Advanced features </span>



<a id='s3.1'></a>

### 3.1 Quantile



- a value between 0 and 1
- shows us the cut-off point for the partition value between a custom bucket

<img src="../../Visualisations/Cut and Quantile.jpg" style="height: 400px;"/>

In [9]:
#Give me the cut-off point for price between 1 and 2nd quintile
index = bq.univ.members('SASEIDX Index')
price = bq.data.px_last(CURRENCY='USD')
price_cutoff = price.group().quantile('0.80')

req = bql.Request(index, {'Cut Off': price_cutoff}) 
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,ORIG_IDS,Cut Off
ID,,,,
IdentityGroup,2020-10-30,USD,None,13.368884


#### 3.1.1 Quantile Practice
##### <span style="color:darkorange">Q </span>: Find the securities within the first quintile ie price > cut-off point


In [30]:
index = bq.univ.members('SASEIDX Index')
price = bq.data.px_last(CURRENCY='USD')
price_cutoff = price.group().quantile('0.80')
#Get the cut-off to a security level
price_cutoff_u = price_cutoff.ungroup()
price_criteria = price > price_cutoff_u
filtered_index = bq.univ.filter(index, price_criteria)

req = bql.Request(filtered_index, {'Price': price}) 
res = bq.execute(req)
data = res[0].df().head()
data

,DATE,CURRENCY,Price
ID,,,
ARABSEA AB Equity,2020-10-30,USD,20.103986
BAAZEEM AB Equity,2020-10-30,USD,13.491535
ALOMRAN AB Equity,2020-10-30,USD,18.717505
ANAAM AB Equity,2020-10-30,USD,20.957206
CATERING AB Equity,2020-10-30,USD,20.743901


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s3.2'></a>

### 3.2. Rolling calculations

- Calculate with a rolling time window over a time period
- Rolling() shift tab to see inputs and parameters

<img src="../../Visualisations/Rolling.jpg" style="width: 600px;"/>

Q: Show me the rolling Earnings momentum for an index over a year with a window of 12 weeks which rolls on a weekly basis

In [11]:
index = bq.univ.members('MXEU Index')
#Momentum for all members
up = bq.data.contributor_revisions(bq.data.is_eps(FPO='1', FPT='A'), revision_type='numup', revision_window='12w')
up_tot = up.group().sum()
down = bq.data.contributor_revisions(bq.data.is_eps(FPO='1', FPT='A'), revision_type='numdn', revision_window='12w')
down_tot = down.group().sum()
count = bq.data.contributor_count(bq.data.is_eps(FPO='0', FPT='A'))
count_tot = count.group().sum()
momentum_tot = (up_tot - down_tot) / count_tot
#Round to 3 decimal places
momentum_tot_round = momentum_tot.round('3')
#Roll for a year on a weekly basis with a window size of 12 weeks
momentum_roll = momentum_tot_round.rolling(ITERATIONDATES=bq.func.range('-1y','0d', frq='w'))

req = bql.Request(index, {'Rolling Momentum':momentum_roll})
res = bq.execute(req)
data = res[0].df().head()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,ORIG_IDS:0,ORIG_IDS:1,ITERATION_DATE,ITERATION_ID,Rolling Momentum
ID,,,,,,,,
IdentityGroup,2019-10-30,2020-08-31,2019-10-30,None,IdentityGroup,2019-10-30,IdentityGroup:ITR0,NaN
IdentityGroup,2019-11-06,2020-09-30,2019-11-06,None,IdentityGroup,2019-11-06,IdentityGroup:ITR1,NaN
IdentityGroup,2019-11-13,2020-09-30,2019-11-13,None,IdentityGroup,2019-11-13,IdentityGroup:ITR2,NaN
IdentityGroup,2019-11-20,2020-09-30,2019-11-20,None,IdentityGroup,2019-11-20,IdentityGroup:ITR3,NaN
IdentityGroup,2019-11-27,2020-09-30,2019-11-27,None,IdentityGroup,2019-11-27,IdentityGroup:ITR4,NaN


#### 3.2.1 Rolling Practice
Find the Index Median Correlation

##### <span style="color:darkorange">Q </span>: What was the rolling median correlation of 1yr total return of UKX members against the index itself?

- Using corr() to calculate correlation of a time series of 1 yr total return against that for an index
- Derive the median correlation for all members against the index
- Apply rolling() to make this computation on a rolling basis between 2018-11-01 and 2018-11-30

In [12]:
index = bq.univ.members('SX5E Index')

ttr = bq.data.day_to_day_total_return(dates=bq.func.range('-1y', '0d'), CURRENCY='EUR')
index_ttr = bq.func.value(ttr, bq.univ.list(['SX5E Index']))
#Calculate correlation of index member total return against index itself
corr = bq.func.corr(ttr, index_ttr)
corr_median = corr.group().median()
#Roll our calculation
corr_median_rolling = bq.func.rolling(corr_median, ITERATIONDATES=bq.func.range('2018-11-01', '2018-11-30'))

req = bql.Request(index, {'med_corr': corr_median_rolling}) 
res = bq.execute(req)
data = res[0].df().head()
data

,DATE,ORIG_IDS:0,ORIG_IDS:1,ITERATION_DATE,ITERATION_ID,med_corr
ID,,,,,,
IdentityGroup,2018-11-01,None,IdentityGroup,2018-11-01,IdentityGroup:ITR0,0.617809
IdentityGroup,2018-11-02,None,IdentityGroup,2018-11-02,IdentityGroup:ITR1,0.616108
IdentityGroup,2018-11-03,None,IdentityGroup,2018-11-03,IdentityGroup:ITR2,0.616545
IdentityGroup,2018-11-04,None,IdentityGroup,2018-11-04,IdentityGroup:ITR3,0.616544
IdentityGroup,2018-11-05,None,IdentityGroup,2018-11-05,IdentityGroup:ITR4,0.616246


</a>[Return to Index ↑](#s0)<a href='#s0'></a>


<a id='s3.3'></a>

### 3.3 Value

- Introduces data from outside the universe
- Allows us to retrieve and manipulate data from two universes in one request
- Be aware that Value() also takes its universe as a bq.univ.list()

<img src="../../Visualisations/BQL Value.jpg" style="width: 600px;"/>

In [13]:
#Value allows us to reference two separate universes
stock = 'ADS GR Equity'
index = 'INDU Index'
#pull price of Adidas and name of Dow Jones in the same query
price = bq.data.px_last()
name = bq.data.name()
name_index = name.value(bq.univ.list([index]))
#Put our fields in an OrderedDict()
flds = OrderedDict()
flds['Adidas Price'] = price
flds['Index Name'] = name_index

req = bql.Request(stock, flds)
res = bq.execute(req)
tbl = pd.DataFrame({r.name:r.df()[r.name] for r in res})
tbl

,Adidas Price,Index Name
ID,,
ADS GR Equity,258.399994,Dow Jones Industrial Average


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s3.4'></a>
### 3.4. Advanced Filtering


- instead of using or() to evaluate multiple conditions within one field, in_() lets BQL do this much quicker
- in_().not_() can be used for negative screening

<img src="../../Visualisations/in.jpg" style="width: 600px;"/>

#### 3.4.1 Method 1: Get the stocks in by sector for the members of the SXXP in Energy, Healthcare, Financials and Industrials using or()

In [14]:
index = bq.univ.members('SXXP Index')
sector = bq.data.classification_name()
criteria = bq.func.or_(sector == 'Industrials', bq.func.or_(sector == 'Financials',bq.func.or_(sector == 'Energy', sector == 'Healthcare')))
filtered_index = bq.univ.filter(index, criteria)
#Count by sector
security = bq.data.id()
security_per_sector = security.group(sector).count()

req = bql.Request(index, {'Securities per Sector': security_per_sector}) 
res = bq.execute(req)
data = res[0].df().head()
data

,Weights,Positions,ORIG_IDS,CLASSIFICATION_NAME(),Securities per Sector
ID,,,,,
Communications,NaN,NaN,SXXP INDEX,Communications,39
Consumer Discretionary,NaN,NaN,SXXP INDEX,Consumer Discretionary,56
Consumer Staples,NaN,NaN,SXXP INDEX,Consumer Staples,53
Energy,NaN,NaN,SXXP INDEX,Energy,21
Financials,NaN,NaN,SXXP INDEX,Financials,102


#### 3.4.2 Method 2: using in_()
- Less nesting, much easier to read and change

In [15]:
index = bq.univ.members('SXXP Index')
sector = bq.data.classification_name()
criteria = sector.in_(['Energy', 'Financials', 'Healthcare','Industrials'])
filtered_index = bq.univ.filter(index, criteria)
#Count by sector
security = bq.data.id()
security_per_sector = security.group(sector).count()

req = bql.Request(index, {'Securities per Sector': security_per_sector}) 
res = bq.execute(req)
data = res[0].df().head()
data

,Weights,Positions,ORIG_IDS,CLASSIFICATION_NAME(),Securities per Sector
ID,,,,,
Communications,NaN,NaN,SXXP INDEX,Communications,39
Consumer Discretionary,NaN,NaN,SXXP INDEX,Consumer Discretionary,56
Consumer Staples,NaN,NaN,SXXP INDEX,Consumer Staples,53
Energy,NaN,NaN,SXXP INDEX,Energy,21
Financials,NaN,NaN,SXXP INDEX,Financials,102


#### 3.4.3 Reversing logic for negative screening
- in_().not_()
- Exclude securities that fulfill the following criteria in a field

<img src="../../Visualisations/not in.jpg" style="width: 600px;"/>

In [16]:
index = bq.univ.members('SXXP Index')
sector = bq.data.classification_name('BICS','4')
criteria = sector.in_(['Alcoholic Beverages', 'Tobacco Products']).not_()
filtered_index = bq.univ.filter(index, criteria)
#Median Sales Growth by Sector
sales_growth = bq.data.sales_growth(FPT='A',FPO='1')
growth_per_sector = sales_growth.group(sector).median()

req = bql.Request(index, {'Securities per Sector': growth_per_sector}) 
res = bq.execute(req)
data = res[0].df().head(30)
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,ORIG_IDS,"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME.BICS,CLASSIFICATION_LEVEL.4)",Securities per Sector
ID,,,,,,
Advertising & Marketing,2020-10-29,2020-12-31,2020-10-30,None,Advertising & Marketing,-15.001937
Agricultural Chemicals,2020-10-30,2020-12-31,2020-10-30,YAR NO Equity,Agricultural Chemicals,-7.575758
Agricultural Machinery,2020-10-27,2020-12-31,2020-10-30,None,Agricultural Machinery,-11.360248
Agricultural Producers,2020-10-16,2021-06-30,2020-10-30,None,Agricultural Producers,4.104890
Aircraft & Parts,2020-10-29,2020-12-31,2020-10-30,None,Aircraft & Parts,-27.760577
Airlines,2020-10-30,2020-12-31,2020-10-30,None,Airlines,-63.502124
Alcoholic Beverages,2020-10-26,2021-03-31,2020-10-30,None,Alcoholic Beverages,-2.724702
"Apparel, Footwear & Acc Design",2020-10-29,2021-03-31,2020-10-30,None,"Apparel, Footwear & Acc Design",-13.812955
Application Software,2020-10-27,2020-12-31,2020-10-30,None,Application Software,0.943660


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s3.5'></a>
### 3.5 Matches()

- Matches performs calculation on a subset of data defined by conditions
- It doesn't screen out results which do not match your criteria, like filter does
- Will derive different results based on the universe which is being analyzed

In [17]:
#Pull Price for Vodafone over the last month
stock = 'IBM US Equity'
px = bq.data.px_last(dates=bq.func.range('-1m','0d')).dropna()

req = bql.Request(stock, {'Price': px}) 
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Price
ID,,,
IBM US Equity,2020-09-30,USD,121.67
IBM US Equity,2020-10-01,USD,121.09
IBM US Equity,2020-10-02,USD,120.57
IBM US Equity,2020-10-05,USD,122.01
IBM US Equity,2020-10-06,USD,121.97
IBM US Equity,2020-10-07,USD,124.07
IBM US Equity,2020-10-08,USD,131.49
IBM US Equity,2020-10-09,USD,127.79
IBM US Equity,2020-10-12,USD,127.21


In [18]:
#Pull Price for Vodafone over the last month over a threshold
stock = 'IBM US Equity'
px = bq.data.px_last(dates=bq.func.range('-1m','0d')).dropna()
px_high = px.matches(px > 150)

req = bql.Request(stock, {'Price High': px_high}) 
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Price High,Partial Errors
ID,,,,
IBM US Equity,NaT,None,NaN,"Error, no match found for identifier."


In [19]:
#This allows you to perform functions on the data item

stock = 'IBM US Equity'
px = bq.data.px_last(dates=bq.func.range('-1m','0d')).dropna()
px_q = px.quantile(0.75)
px_top = px.matches(px > px_q)

req = bql.Request(stock, {'Price High': px_top}) 
res = bq.execute(req)
data = res[0].df()
data


,DATE,CURRENCY,Price High
ID,,,
IBM US Equity,2020-10-08,USD,131.49
IBM US Equity,2020-10-09,USD,127.79
IBM US Equity,2020-10-12,USD,127.21
IBM US Equity,2020-10-14,USD,125.94
IBM US Equity,2020-10-16,USD,125.93
IBM US Equity,2020-10-19,USD,125.52


Matches is perfect when you need multiple calculations performed on the same universe

##### <span style="color:darkorange">Q </span>: Let's get the total return for stocks since the date of their lowest cumulative return

In [20]:
index = bq.univ.members('SMI Index')
#let's calculate cumulative return
tot_ret_time_series = bq.data.day_to_day_tot_return_gross_dvds(start='-1y', end='0d')
cum_ret = (1 + tot_ret_time_series).ln().cumsum().exp()
#Find the date of the minimum Cumulative return
min_ret = cum_ret.min()
#This finds the date column of the min return
min_ret_date = min_ret['DATE']
#Find all the returns with a later date than the date of the minimum cumulative return
cum_ret_since_min = bq.func.matches(cum_ret, cum_ret['DATE'] >= min_ret_date)
#Calculate the latest relative return relative to the minimum cum return during this period
tot_ret_adj = cum_ret_since_min/min_ret
tot_ret_adj_latest = tot_ret_adj.last('1')

req = bql.Request(index, {'Total Cum Return Since Minimum': tot_ret_adj_latest}) 
res = bq.execute(req)
data = res[0].df()
data

,DATE,Total Cum Return Since Minimum
ID,,
ABBN SE Equity,2020-10-30,1.563162
CFR SE Equity,2020-10-30,1.184096
CSGN SE Equity,2020-10-30,1.329279
GEBN SE Equity,2020-10-30,1.392957
GIVN SE Equity,2020-10-30,1.430906
LHN SE Equity,2020-10-30,1.427471
LONN SE Equity,2020-10-30,1.727071
NESN SE Equity,2020-10-30,1.168399
NOVN SE Equity,2020-10-30,1.017582


<img src="../../Visualisations/Matches 2.jpg" style="width: 800px;"/>

##### <span style="color:darkorange">Q </span>: What was the Z-Score of PE Ratios for the DOW Jones where the ZScore was greater than 1.5?

In [31]:
univ = bq.univ.members('indu index')
pe = bq.data.pe_ratio()
pe_zscore = pe.groupzscore()
# screening with filter
filtered_univ = bq.univ.filter(univ, pe_zscore>1.5)
req = bql.Request(filtered_univ,pe_zscore )
res = bq.execute(req)
data = res[0].df()
data

,AS_OF_DATE,REVISION_DATE,PERIOD_END_DATE,ORIG_IDS,IDENTITYGROUP,GROUPZSCORE(PE_RATIO())
ID,,,,,,
CRM UN Equity,2020-10-30,2020-08-28,2020-07-31,CRM UN Equity,IdentityGroup,NaN


The results for our screen do not match our criteria because:
- We screened our universe for Zscore greater than 1.5
- The results showed us the Z-score of PE for the Z-Scores that were above 1.5 (Z-Score was calculated on two levels)
- To see the results we asked for the first time, we need matches()

In [32]:
univ = bq.univ.members('indu index')
pe = bq.data.pe_ratio()
pe_zscore = pe.groupzscore()
#screening with matches
filterd_pe = bq.func.matches(pe_zscore,pe_zscore>1.5).dropna(remove_id = True)
req = bql.Request(univ, filterd_pe)
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,ACT_EST_DATA,AS_OF_DATE,ORIG_IDS,"DROPNA(MATCHES(GROUPZSCORE(PE_RATIO()),(GROUPZSCORE(PE_RATIO())>1.5)),remove_id=true)"
ID,,,,,,
CRM UN Equity,2020-08-28,2020-07-31,A,2020-10-30,None,5.194217


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4'></a>

<span style="color:darkorange; font-size:2em"> 4 Scoring model </span>
<br>

Let's apply everything we learnt and apply it to an actual situation
- Different ways to normalize data for peer comparison
  - Weighting of fundamental data
  - use if_() statements directly on BQL to standardize logic

In [33]:
#Considering the repetition of inputs, it is helpful to create a dictionary as a shortcut for parameters:
current = {'FPO':'0', 'FPT':'A'}
prev = {'FPO':'-1', 'FPT':'A'}
#shortening func and data
f = bq.func
d = bq.data

</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4.1'></a>

### 4.1 Piotroski F-Score

<img src="../../Visualisations/F-Score.jpg" style="width: 600px;"/>

What is the distribution of quality across sectors of S&P 500 Index?

- Use if_() to create scores of 1 or 0 based on custom criteria
- Combine with group() to aggregate scoring metrics to generate an overview across the universe
- https://en.wikipedia.org/wiki/Piotroski_F-Score

In [34]:
#Universe
index = bq.univ.members('SXXP Index')
sector = bq.data.classification_name()

roa = f.if_( d.return_on_asset(**current) > 0 , 1, 0 )
ocf = f.if_( d.cf_cash_from_oper(**current) > 0, 1, 0 )
roa_chg = f.if_( d.return_on_asset(**current) > d.return_on_asset(**prev) , 1, 0 )
acc = f.if_( (d.cf_cash_from_oper(**current) / d.bs_tot_asset(**current)) > d.return_on_asset(**current), 1, 0 )
lvg_chg = f.if_( d.lt_debt_to_tot_asset(**current) < d.lt_debt_to_tot_asset(**prev), 1, 0 )
cr_chg = f.if_( d.cur_ratio(**current) > d.cur_ratio(**prev), 1, 0 )
shs_chg = f.if_( d.bs_sh_out(**current) > d.bs_sh_out(**prev), 1, 0 )
gm_chg = f.if_( d.gross_margin(**current) > d.gross_margin(**prev), 1, 0 ) 
asset_turn = f.if_( d.asset_turnover(**current) > d.asset_turnover(**prev), 1, 0 )
fscore = roa + ocf + roa_chg + acc + lvg_chg + cr_chg + shs_chg + gm_chg + asset_turn

fscore_avg = fscore.group(sector).avg()

req = bql.Request(index, {'Average F-Score Per Sector': fscore_avg}) 
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS,CLASSIFICATION_NAME(),Average F-Score Per Sector
ID,,,
Communications,None,Communications,3.923077
Consumer Discretionary,None,Consumer Discretionary,3.625000
Consumer Staples,None,Consumer Staples,4.377358
Energy,None,Energy,3.809524
Financials,None,Financials,3.843137
Health Care,None,Health Care,4.471698
Industrials,None,Industrials,4.279279
Materials,None,Materials,3.816667
Real Estate,None,Real Estate,4.394737


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4.2'></a>

### 4.2 Beneish M-Score

How can you analyze companies likely to be earnings manipulators based on the Beneish-M?
- Calculate changes in calculated fundamental factors
- M Score below -2.22 unlikely to manipulate earnings, above - 1.78 probable manipulator
- https://en.wikipedia.org/wiki/Beneish_M-Score
- Combine with filter() to narrow down on securities worth worth analysis

<img src="../../Visualisations/M Score.jpg" style="width: 600px;"/>

In [35]:
index = bq.univ.members('SZCOMP Index')
#define our factors to be used
DSRI = f.znav( ( d.sales_rev_turn(**current) / d.bs_acct_note_rcv(**current) ) / ( d.sales_rev_turn(**prev) / d.bs_acct_note_rcv(**prev) ) )
GMI = f.znav( ( d.gross_profit(**current) / d.sales_rev_turn(**current) ) / ( d.gross_profit(**prev) / d.sales_rev_turn(**prev) ) )
AQI = f.znav( ( (d.bs_tot_asset(**current) - d.bs_cur_asset_report(**current) - d.bs_net_fix_asset(**current)) / d.bs_tot_asset(**current) ) / ( (d.bs_tot_asset(**prev) - d.bs_cur_asset_report(**prev) - d.bs_net_fix_asset(**prev))/d.bs_tot_asset(**prev)) )
SGI = f.znav( d.sales_rev_turn(**current) / d.sales_rev_turn(**prev) )
DEPI = f.znav( (d.cf_depr_amort(**prev) / (d.cf_depr_amort(**prev) + d.bs_net_fix_asset(**prev)) ) / (d.cf_depr_amort(**current) / (d.cf_depr_amort(**current) + d.bs_net_fix_asset(**current))) )
SGAI = f.znav( (d.is_cogs_to_fe_and_pp_and_g(**current)/d.sales_rev_turn(**current))/(d.is_cogs_to_fe_and_pp_and_g(**prev)/d.sales_rev_turn(**prev)) )
TATA = f.znav( (d.is_inc_bef_xo_item(**current) - d.cf_cash_from_oper(**current))/d.bs_tot_asset(**current) )
LVGI = f.znav( (d.bs_tot_asset(**current) / ( d.bs_lt_borrow(**current) + d.bs_st_borrow(**current) + d.bs_cur_liab(**current) )) / ( d.bs_tot_asset(**prev) / ( d.bs_lt_borrow(**prev) + d.bs_st_borrow(**prev) + d.bs_cur_liab(**prev) )) )

mscore = -4.84 + 0.92 * DSRI + 0.528 * GMI + 0.404 * AQI + 0.892 * SGI + 0.115 * DEPI - 0.172 * SGAI + 4.679 * TATA - 0.327 * LVGI
mscore_filter = mscore > -1.78
manipulators_likely = bq.univ.filter(index, mscore_filter)
mscore_top = mscore.grouprank() <= 10
manipulators_top = bq.univ.filter(manipulators_likely, mscore_top)

req = bql.Request(manipulators_top, {'M-Score': mscore.groupsort()}) 
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,M-Score
ID,,,,,
000985 CH Equity,2020-04-18,2019-12-31,2020-10-30,None,inf
000752 CH Equity,2020-04-28,2019-12-31,2020-10-30,None,inf
000567 CH Equity,2020-04-26,2019-12-31,2020-10-30,None,inf
000953 CH Equity,2020-04-28,2019-12-31,2020-10-30,None,4414.653865
000982 CH Equity,2020-04-28,2019-12-31,2020-10-30,None,206.753624
300029 CH Equity,2020-04-28,2019-12-31,2020-10-30,None,130.819936
000987 CH Equity,2020-02-28,2019-12-31,2020-10-30,None,25.994627
000996 CH Equity,2020-04-30,2019-12-31,2020-10-30,None,24.511451
002210 CH Equity,2020-06-30,2019-12-31,2020-10-30,None,23.969235


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4.3'></a>

### 4.3 Scoring Practice

Create your own custom factor models using Z-Scores and Matches()
- Z-Score normalizes datapoints by measuring no. standard deviations from the mean
- Matches allows us to apply multiple overlapping calculations and drill down to the results

<img src="../../Visualisations/Matches 3.jpg" style="width: 800px;"/>

In [36]:
index = bq.univ.members('SXXP Index')
#Define your own custom factors
#BQL allows you to build custom technical analysis analysis
momentum = bq.data.rsi(close=bq.data.px_last(), period='222').znav()
#Use BQL's custom return calculations
profitability = bq.data.alpha(calc_interval=bq.func.range('-252d','0d'), benchmark_ticker='SXXP Index').znav()
volatility = bq.data.volatility(calc_interval='252d').znav()
value = 0.5* (1/bq.data.px_to_book_ratio()) + 0.5 * (bq.func.avail(bq.data.ebitda(),bq.data.is_oper_inc())/bq.data.curr_entp_val())
value = value.znav()
#Calculate the Z-scores
momentum_z = momentum.groupzscore()
profit_z = profitability.groupzscore()
vol_z = volatility.groupzscore()
value_z = value.groupzscore()
#Add up your scores
final_score = momentum_z + profit_z + vol_z + value_z
#Let's see the top 30% of scores
top_30 = bq.func.matches(final_score, final_score> final_score.group().quantile('0.70').ungroup())
#Clean our results to remove the rows which did not match our criteria
top_30_clean = top_30.dropna(remove_id='True')

req = bql.Request(index, {'Final Score': top_30_clean.round('3').groupsort()})
res = bq.execute(req)
data = res[0].df().head()
data

,CURRENCY,Final Score
ID,,
WMH LN Equity,None,9.454
AGN NA Equity,None,7.741
RNO FP Equity,None,7.534
RMG LN Equity,None,5.578
CCL LN Equity,None,5.154


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s5'></a>

<span style="color:green; font-size:2em"> 5. BQL Equity Advanced Summary </span>



### BQL

- Custom calculations: combining fields, applying BQL functions, using new BQL metrics
- Rolling calculations to analyse momentum
- Advanced filtering in a more efficient manner and apply negative logic
- Apply scoring methodology to normalize combinations of fields and screen

### Benefits
- Save time and effort cleaning, manipulating, sorting data
- Data-efficient analysis
- Focus on generating ideas and alpha

<span style="color:yellow; font-size:2em"> Congratulations - What's Next? </span>

<img src="../../Visualisations/BQL Services.jpg" style="width: 400px;"/>

- This is the beginning: Use BQL to dig deep and conduct in-depth cross-asset financial data analysis
- Regular Updates for future releases: Webinars, BQNT Spotlight, BQNT Whatsnew
- Connection to other Bloomberg services, eg Portfolio integration (PORT), order management systems (AIM, TOMS), proprietary data (CDE)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="2.1 Equity Basics.ipynb">&larr; Back to the Equity Basics</a>&emsp;&emsp;
    <a href="#s0">&uarr; Back to Top </a>&emsp;&emsp;
    <a href="../../0 Welcome.ipynb">Back to Home &rarr;</a>
    <br>

</p>